In [1]:
from Crypto import *

In [51]:
# uses ppm in ascii format, not bin
def keyFromCat(file):
    with open(file, 'r') as f:
        contents = f.readlines() # num pixels * 3 + 4

        (width, height) = [int(i) for i in contents[2].split(" ")]
        assert(int(contents[3]) == 255)
        max_length = width*height-20

        # we need key length
        # we'll start from rc4, that seems the easiest to implement
        # let's say keys can be 1000 somethings long, so +-5 on each rgb value

        lenny = (int(contents[4]) % 10) * 100 + (int(contents[5]) % 10) * 10 + (int(contents[6]) % 10)
        assert(max_length > lenny)
        print(f"Max length: {max_length}, actual length: {lenny}")

        #key_in_int = [int("0o" + str(int(contents[i]) % 8) + str(int(contents[i+1]) % 8) + str(int(contents[i+2]) % 8), base=8) for i in range(7, 7+lenny*3, 3)]
        key_in_int = [int(contents[i]) % 16 for i in range(7, lenny)]
        if (t := len(key_in_int)) < 16:
            ValueError("try a bigger image")
        elif t < 24:
            return bytes(key_in_int[:16])
        elif t < 24:
            return bytes(key_in_int[:24])
        else:
            return bytes(key_in_int[:32])

print(keyFromCat("Cat.ppm"))

Max length: 1332980, actual length: 259
b'\n\x03\r\t\x02\x0c\x08\x01\x0b\x06\x0f\t\x05\x0e\x08\x04\r\x07\x03\x0c\x06\x00\x0b\x05\x00\x0b\x05\x0e\t\x03\r\x08'


In [48]:
from Crypto.Cipher import AES
def encryptFile(file, key):
    # match for different encryption schemes
    cipher = AES.new(key, AES.MODE_EAX)
    nonce = cipher.nonce

    with open(file, 'rb') as f:
        ciphertext, tag = cipher.encrypt_and_digest(f.read())
    return (nonce, ciphertext, tag)

In [54]:
file = "html_central.html"
nonce, encrypted, tag = encryptFile(file, keyFromCat("Cat.ppm"))
with open(file+".kky", 'wb') as f:
    f.write(encrypted)
cipher = AES.new(keyFromCat("Cat.ppm"), AES.MODE_EAX, nonce=nonce)
print(cipher.decrypt(encrypted))

Max length: 1332980, actual length: 259
Max length: 1332980, actual length: 259
b'<!DOCTYPE html>\r\n<head>\r\n    <title> Gru? </title>\r\n    <link type=\'text/css\' href=\'central.css\' rel=\'stylesheet\' />\r\n</head>\r\n<body>\r\n    <div class = "body_content">\r\n        <div class = "Intial-Text">\r\n            <h1>Here you will find a collection of astronomical objects. All desireable for some petty little thiefs.</h1>\r\n        </div>\r\n        <div>\r\n            <p id = "HiddenText"> Maybe think of someone with a lot of MINIONS...</p>\r\n        </div>\r\n        <div class = "central-items" >\r\n            <img id = "central-image" , src = "full-moon-moon-bright-sky-47367.jpeg">\r\n            <div class = "Input-buttons">\r\n                <input class = "inputs" id = "Astro_Search">\r\n                <div class = "buttons">\r\n                    <div id = "Search"> Search the System</div>\r\n                    <div id = "Random"> Random Catcher</div>\r\n        